In [ ]:
import pandas as pd
import requests as rq
import json
import urllib.parse as urlparse
from urllib.parse import parse_qs
from urllib.parse import unquote
from datetime import datetime

In [ ]:
# Helper method for extracting the number of the next skip value
def extract_skip(url):
    decoded = unquote(url)
    parsed = urlparse.urlparse(decoded)
    result = parse_qs(parsed.query)['$skip'][0]
    return int(result)

def read_page(items, url):
    reponse = rq.get(url)
    page = json.loads(reponse.text)
    items += page['Items'] 
    next_page = page['NextPageLink']
    return items, next_page

items = []   

In [ ]:
url = "https://prices.azure.com/api/retail/prices?currencyCode='USD'&$filter=serviceFamily%20eq%20'Compute'"
items, next_page = read_page(items, url)

In [ ]:
while next_page:
    items, next_page = read_page(items, next_page)    
    if not next_page:
        break
    
    # only print for every 1000 pages
    skip = extract_skip(next_page)
    if skip % 1000 == 0 :
        now = datetime.now()
        dt_string = now.strftime('%H:%M:%S')
        print (dt_string, ': ', next_page)
        
print('===== Done ======')

In [ ]:
# pickle
json_data = pd.DataFrame(items)
json_data.to_pickle('./data/azure.xz.pkl', compression='xz')